<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFEpostprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**pip**
---



In [2]:
%%time
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
!pip install memory_profiler

CPU times: user 405 ms, sys: 87.1 ms, total: 492 ms
Wall time: 1min 55s


**import**
---



In [0]:
import glob
import os
import pathlib as Path
from sys import getsizeof
import dask
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
from dask.distributed import Client
from shapely.geometry import box

In [0]:
%%capture
client = Client(processes=False)
client

In [12]:
%cd /content/drive/My Drive/Colab Notebooks/01_githubRepo/FFE

/content/drive/My Drive/Colab Notebooks/01_githubRepo/FFE


In [0]:
pathShapefile = './data/shapefile'
path_output = '../../02_output'

In [23]:
!ls '../../02_output'

scenario1249_pid958_Activations.parquet  smallShapeEdges.parquet
scenario1249_pid959_Activations.parquet


**definitions**
---

In [0]:
@dask.delayed
def read_and_concatenate_parquets(prefix, path=path_output):
  L = []
  files = glob.glob(os.path.join(path_output, prefix))# output_scenario_0_step_0.parquet
  for file in files:
    print("file loaded : {}".format(file))
    pqt = dd.read_parquet(file)
    L.append(pqt)
  df = dd.concat(L)
  return df

def count_fid_occurences(df):
  count = df['source'].value_counts().compute()
  count_df = pd.DataFrame({'source': count.index, 'count': count.values})
  return count_df

def load_shapefile(file_name, minx, miny, maxx, maxy):
    # crop data
    bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(pathShapefile,file_name), bbox=bbox)
    max_extent = gdf_buildings.total_bounds
    data_size = getsizeof(gdf_buildings) /(1024.0**3)
    print("Shapefile extent : {}".format(max_extent))
    print("Asset loaded : {}".format(len(gdf_buildings)))
    # gdf_buildings.plot(column='IgnProb_bl', cmap='hsv', legend=True)
    return gdf_buildings

def merge_coordinates_export_shape(ddf, gdf, name_output):
  gdf = gdf[['TARGET_FID', 'geometry']]
  df = pd.DataFrame(gdf)
  # ddf = ddf.compute()
  df_merge = ddf.merge(df, how='left', left_on='source', right_on='TARGET_FID')
  gdf_merge = gpd.GeoDataFrame(df_merge, geometry='geometry')
  gdf_merge.to_file(os.path.join(path_output, name_output + ".shp"))

  f, ax = plt.subplots(1, figsize=(12, 12))
  countPlot = gdf_merge.plot(column='count', cmap='seismic', legend=True, ax=ax)
  plt.savefig(os.path.join(path_output, 'smallMonteCarlo2500runs.jpg'), dpi=300)
  plt.show()
  return gdf_merge

**runs**
---



In [0]:
df = read_and_concatenate_parquets("scenario*")

In [0]:
t.to_parquet(os.path.join(path_output, 'smallMonteCarlo2500runs.parquet'), engine='pyarrow')

In [49]:
count_df = count_fid_occurences(df)

file loaded : ../../02_output/scenario1249_pid958_Activations.parquet
file loaded : ../../02_output/scenario1249_pid959_Activations.parquet


In [0]:
count_df.describe()

In [0]:
gdf = load_shapefile("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small
gdf_count = merge_coordinates_export_shape(count_df, gdf, "smallBurnedBuildings")